In [28]:
import random
import numpy as np

B = 3                    #tipos de veiculos b
R = 4                     #conjunto de rotas r
FT = [65,40,55]            #frota disponível por tipo de veiculo b
PD = [20,30,45,60]           #demanda de passageiros por hora na rota r
cap = [80,80,80]               #capacidade de passageiros para cada tipo de veiculo b
frequencia = np.zeros((B,R))    #frequência de serviço do tipo de veículo b na rota r 
Q = np.zeros((B,R))              #número de passageiros alocados em cada rota r pelos diferentes tipos de veículos b

CO = [4.05, 4.20, 4.80]     #custo operacional (motorista, gasolina e manutenção) para cada veículo do tipo b
CE = 0.12                  #custo de espera na rota r
CD = 0.18                 #custo de desconforto para passageiros     
T = 60                  #período de planejamento (60 min)
H_br = np.zeros((B,R))   #intervalo de tempo entre as partidas do tipo de veículo b na rota r 

rotas = []                        #lista de rotas restantes
S = 10                              #número de soluções geradas

#imprimindo os parâmetros
print("\n Rotas = ", R, "\n Tipo de veiculo = ", B)
print(" Frota = ", FT, "\n Demanda de passageiros = ", PD, "\n Capacidade do veiculo = ", cap, "\n")
print(" Custo Operacional = ", CO, "\n Custo de Espera = ", CE, "\n Custo de Desconforto = ", CD, "\n Periodo = ", T, "\n")

#gerando as soluções
#para cada solução s
for s in range(S): 
    #variável auxiliar p(passageiros) recebe a demanda de passageiros por hora na rota r
    p = PD.copy()
    #variável auxiliar c(capacidade) recebe a capacidade de passageiros para cada tipo de veiculo b
    c = cap.copy()
    
    #para cada tipo de veículo b
    for b in range(B):
        #número máximo de veículos disponíveis para cada tipo de veículo b 
        max_v = FT[b]
        #para cada rota r
        for r in range (R):
            #adiciona o conjunto de rotas na lista de rotas restantes
            rotas.append(r)
        
        #enquanto houver rotas restantes
        while(rotas):
            #sorteia uma rota r
            r = random.choice(rotas)  
            #pega um valor que esteja entre a capacidade de passageiros e a demanda de passageiros
            min_value = min(c[b],p[r])
            #sorteia um número de veículo enviado para a rota r 
            frequencia[b][r] = random.randint(0,max_v)
            #se nenhum veículo foi enviado para rota r, 
            if frequencia[b][r] == 0:
                #a demanda de passageiros será igual a zero
                Q[b][r] = 0
                #e o intervalo de tempo entre as partidas do tipo de veículo b na rota r, também será igual a zero
                H_br[b][r]=0
                
                #remove a rota r da lista de rotas restantes
                rotas.remove(r)
                continue
                
            #se o número de veículo enviado para rota r for diferente de zero,
            #então calcula-se o H_br (intervalo de tempo entre as partidas do tipo de veículo b na rota r)
            H_br[b][r] = T/frequencia[b][r]
            
            #sorteia um número de passageiros para cada tipo de veículo b na rota r
            Q[b][r] = random.randint(0, min_value)            
            #atualiza a capacidade de passageiros b
            c[b] -= Q[b][r]
            #atualiza a demanda de passageiros da rota r
            p[r] -= Q[b][r]
            #atualiza o número maximo de veículos disponíveis para cada tipo b 
            max_v -= frequencia[b][r]
            #remove a rota r da lista de rotas restantes
            rotas.remove(r)
        
            
    #imprimindo as soluções
    print("_"*48)
    print("Solucao ", s+1, "\n")
    #para cada tipo de veículo b
    for b in range(B):
        print("_Tipo ", b)
        #para cada rota r
        for r in range(R):
            print("rota ", r, ": ", "qntd_veiculo: ", frequencia[b][r],"  -  qntd_passageiros: ", Q[b][r] )
        print()
  
       






 Rotas =  4 
 Tipo de veiculo =  3
 Frota =  [65, 40, 55] 
 Demanda de passageiros =  [20, 30, 45, 60] 
 Capacidade do veiculo =  [80, 80, 80] 

 Custo Operacional =  [4.05, 4.2, 4.8] 
 Custo de Espera =  0.12 
 Custo de Desconforto =  0.18 
 Periodo =  60 

________________________________________________
Solucao  1 

_Tipo  0
rota  0 :  qntd_veiculo:  20.0   -  qntd_passageiros:  7.0
rota  1 :  qntd_veiculo:  29.0   -  qntd_passageiros:  22.0
rota  2 :  qntd_veiculo:  10.0   -  qntd_passageiros:  43.0
rota  3 :  qntd_veiculo:  2.0   -  qntd_passageiros:  7.0

_Tipo  1
rota  0 :  qntd_veiculo:  21.0   -  qntd_passageiros:  9.0
rota  1 :  qntd_veiculo:  4.0   -  qntd_passageiros:  3.0
rota  2 :  qntd_veiculo:  3.0   -  qntd_passageiros:  0.0
rota  3 :  qntd_veiculo:  12.0   -  qntd_passageiros:  47.0

_Tipo  2
rota  0 :  qntd_veiculo:  31.0   -  qntd_passageiros:  2.0
rota  1 :  qntd_veiculo:  7.0   -  qntd_passageiros:  5.0
rota  2 :  qntd_veiculo:  9.0   -  qntd_passageiros:  0.0
ro